# PRIMAP-hist data Preparation

This jupyter notebook sets out a method for preparing PRIMAPhist data for plotting. It can further be used as a guide and basis for setting up other input datasets. 

The dataset can be viewed and accessed at:
https://www.pik-potsdam.de/paris-reality-check/primap-hist/

Gütschow, J.; Jeffery, L.; Gieseke, R. (2019): The PRIMAP-hist national historical emissions time series (1850-2016). v2.0. GFZ Data Services. https://doi.org/10.5880/pik.2019.001

The downloaded data should be stored in the 'input-data' folder of this repository. PRIMAP-hist is updated roughly once per year. The user should therefore check for updates at the above link and download new data when available. If this is done, the new filenames will need to be added below. 

For further description on the data requirememts, please see **TODO**. 

------
TEMPORARY TO DO LIST!
* consider adding unit conversion
* try a few more examples - is the metadata / data description sufficient?
* ? warn of over-writing files AND/OR automise file names.  Currently it's possible to give a file a poor name. Would require a look-up for category names though...


In [4]:
# import modules

# system
import re
import sys
import os

# data handling
import pandas as pd
import numpy as np

# open climate data packages
from countrygroups import UNFCCC, EUROPEAN_UNION, ANNEX_ONE, NON_ANNEX_ONE
from shortcountrynames import to_name

# global stocktake tools
import gst_tools.gst_utils as utils

In [2]:
## User definition of requirements

include_extrapolated_data = False

# to understand the codes to use here, please see the PRIMAP-hist documentation
raw_entity = 'CO2'
raw_sector = 'IPC1'
raw_scenario = 'HISTCR' 

# choose something useful! These will be used to generate the new filename.
new_variable_name = 'Energy-CO2'
new_source_name = 'PRIMAP-hist_v2.0'

# Based on countrygroups package, select the group of countries you would like to extract. 
# Note that the raw data also includes groups.
needed_countries = UNFCCC

# First year of data needed for further plotting
start_year = 1990 


In [5]:
## reduce the data

# select the right file 
# !*** Note that the file names here will need to be updated for new versions of PRIMAP-hist ***! 
if include_extrapolated_data:
    raw_data_file = 'PRIMAP-hist_v2.0_11-Dec-2018.csv'
else:
    raw_data_file = 'PRIMAP-hist_v2.0_no_extrapolation_11-Dec-2018.csv'
    

# get the data
raw_data_folder = 'input-data'
fname = os.path.join('', raw_data_folder, raw_data_file)
print('reading ' + fname)
raw_data = pd.read_csv(fname)

# reduce to only the desired variable (one per output file)
new_data = raw_data.loc[(raw_data['entity'] == raw_entity) & 
                        (raw_data['scenario'] == raw_scenario) &
                        (raw_data['category'] == raw_sector)
                       ]

# reduce the countries or regions to only those desired
new_data = new_data.loc[raw_data['country'].isin(needed_countries)]

# tell the user if any of the needed countries are missing and, if yes, which ones:
missing_countries = list(set(needed_countries) - set(new_data['country'].unique()))
if missing_countries:
    print('Not all countries requested were available in the raw data. You are missing the following:')
    for country in missing_countries:
        print('   ' + to_name(country))
    print('---------')


# reduce to only required years
new_data = utils.change_first_year(new_data, start_year)

# rename columns to follow conventions
new_data = new_data.rename(columns={'entity': 'variable'})

new_data['source'] = new_source_name

new_data = utils.check_column_order(new_data)

# check! 
new_data

reading input-data/PRIMAP-hist_v2.0_no_extrapolation_11-Dec-2018.csv
Not all countries requested were available in the raw data. You are missing the following:
   European Union
   Palestine
---------
Last year of data available is 2016


,category,country,scenario,source,unit,variable,1990,1991,1992,1993,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,IPC1,AFG,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,2560.00,2380.00,1330.00,1290.00,...,2260.0,4190.0,6750.0,8440.00,12200.00,10700.0,9990.0,11100.0,11800.0,14000.0
1,IPC1,AGO,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,4990.00,4960.00,5050.00,5650.00,...,24500.0,24800.0,26900.0,28300.00,29600.00,32600.0,31600.0,33500.0,19900.0,20800.0
2,IPC1,ALB,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,3070.00,4920.00,3100.00,3000.00,...,3930.0,3980.0,4320.0,4460.00,4910.00,4310.0,4420.0,5220.0,4530.0,4760.0
3,IPC1,AND,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,389.00,396.00,403.00,414.00,...,553.0,553.0,530.0,529.00,501.00,500.0,486.0,471.0,NaN,NaN
4,IPC1,ARE,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,43700.00,47800.00,48700.00,55300.00,...,150000.0,172000.0,186000.0,179000.00,184000.00,198000.0,191000.0,239000.0,257000.0,264000.0
5,IPC1,ARG,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,100000.00,109000.00,113000.00,113000.00,...,167000.0,171000.0,167000.0,173000.00,182000.00,182000.0,185000.0,186000.0,185000.0,185000.0
6,IPC1,ARM,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,21000.00,19600.00,24600.00,9680.00,...,4570.0,5200.0,4270.0,4230.00,5010.00,5820.0,5620.0,5660.0,6460.0,6590.0
7,IPC1,ATG,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,282.00,268.00,264.00,271.00,...,469.0,480.0,510.0,524.00,513.00,524.0,524.0,537.0,556.0,566.0
8,IPC1,AUS,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,259000.00,261000.00,266000.00,270000.00,...,374000.0,379000.0,386000.0,381000.00,378000.00,384000.0,376000.0,372000.0,381000.0,392000.0
9,IPC1,AUT,HISTCR,PRIMAP-hist_v2.0,Gg,CO2,51700.00,55400.00,50800.00,51100.00,...,59500.0,59000.0,55600.0,58700.00,56300.00,54400.0,54100.0,50300.0,52100.0,53100.0


In [4]:
## write the data to file

"""
First ensure that years, unit, 'country', and variable are all in data. If they are
can proceed to print data
"""

if 'country' not in new_data.columns or 'unit' not in new_data.columns:
    
    print('Missing required information! Please check your input data and processing!')
    
else:
    
    # define filename as composite of variable and source name
    fname_out = new_source_name + '_' + new_variable_name + '.csv' 
    fullfname_out = os.path.join('proc-data', fname_out)

    # check folder exists
    if not os.path.exists('proc-data'):
        os.makedirs('proc-data')

    # write to csv in proc data folder
    new_data.to_csv(fullfname_out, index=False)

    # celebrate success 
    print('Processed data written to file! - ' + fullfname_out)
    

Processed data written to file! - proc-data/PRIMAP-hist_v2.0_Energy-CO2.csv


**Test ground below**

In [6]:
new_data.columns


Index(['category', 'country', 'scenario', 'source', 'unit', 'variable', '1990',
       '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')